In [22]:
from qiskit import QuantumCircuit
import numpy as np

In [ ]:
def grover_oracle(n, marked_state):
    oracle = QuantumCircuit(n)
    # Flip qubits for bits that are '0' in the marked state.
    for i, bit in enumerate(reversed(marked_state)):
        if bit == '0':
            oracle.x(i)
    # Apply a multi-controlled Z gate.
    if n == 1:
        oracle.z(0)
    else:
        oracle.h(n-1)
        oracle.mcx(list(range(n-1)), n-1)  # multi-controlled Toffoli gate
        oracle.h(n-1)
    # Revert the X gates.
    for i, bit in enumerate(reversed(marked_state)):
        if bit == '0':
            oracle.x(i)
    # Convert the oracle circuit into a gate.
    oracle_gate = oracle.to_gate()
    oracle_gate.name = "Oracle"
    decomposed_circuit = oracle_gate.decompose(recurse=True)
    print(decomposed_circuit.qasm())  # Check that no "Oracle" instruction remains
    return oracle_gate

In [ ]:
def diffuser(n):
    qc = QuantumCircuit(n)
    qc.h(range(n))
    qc.x(range(n))
    # Apply a multi-controlled Z gate
    qc.h(n-1)
    qc.mcx(list(range(n-1)), n-1)  # multi-controlled Toffoli gate
    qc.h(n-1)
    qc.x(range(n))
    qc.h(range(n))
    diffuser_gate = qc.to_gate()
    diffuser_gate.name = "Diffuser"
    return diffuser_gate

In [ ]:
from qiskit_aer import Aer
from qiskit import transpile
n = 3  # number of qubits
marked_state = '101'  # The state we want to search for

# Create a quantum circuit with n qubits and n classical bits.
grover_circuit = QuantumCircuit(n, n)

# Step 1: Initialize all qubits in equal superposition.
grover_circuit.h(range(n))

# Calculate the optimal number of Grover iterations.
iterations = int(np.floor(np.pi/4 * np.sqrt(2**n)))
print("Number of iterations:", iterations)

# Step 2: Apply Grover iterations.
for _ in range(iterations):
    grover_circuit.append(grover_oracle(n, marked_state),range(n))
    grover_circuit.append(diffuser(n), range(n))

# Step 3: Measure all qubits.
grover_circuit.measure(range(n), range(n))

# Draw the circuit.
grover_circuit.draw(output='mpl')

# Run the circuit on the qasm simulator.
backend = Aer.get_backend('qasm_simulator')
job = backend.run(grover_circuit, backend, shots=1024)
result = job.result()
counts = result.get_counts()

print("Measurement counts:", counts)


Number of iterations: 2


AttributeError: 'Gate' object has no attribute 'decompose'

In [3]:
print(2**3**2)

512


In [ ]:
decomposed_circuit = grover_circuit.decompose(recurse=True)
print(decomposed_circuit.qasm())  # Check that no "Oracle" instruction remains